In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/1.17._Fragment_of_donors_prayer.jpg
/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/1.34._Graffiti_names_and_prayers_A.jpg
/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/1.19._Prayer_of_Asterios_B.jpg
/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/1.405._Prayer.jpg
/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/1.12._Prayer_of_Theodoretos_and_Kyriakos_A.jpg
/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/1.12._Prayer_of_Theodoretos_and_Kyriakos_B.jpg
/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/5.122._Donor_prayer_from_a_church.jpg
/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/11.516._Text_at_an_entrance.jpg
/kaggle/input/epigraphie-grecque/data_clean/Prayers + prayer invocation/1.20._Prayer_of_Theodoros.jpg
/kaggle/input/epigraphie-grecque/d

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

DOSSIER_SOURCE = "/kaggle/input/epigraphie-grecque/data_clean"       # ← à adapter
DOSSIER_DEST = "split"   # ← ex. ./dataset_split
RATIO_TEST = 0.2                                  # 20% pour test

# 1. Rassembler tous les fichiers par classe
fichiers = []
labels = []

for classe in os.listdir(DOSSIER_SOURCE):
    chemin_classe = os.path.join(DOSSIER_SOURCE, classe)
    if os.path.isdir(chemin_classe):
        for nom_fichier in os.listdir(chemin_classe):
            chemin_fichier = os.path.join(chemin_classe, nom_fichier)
            if os.path.isfile(chemin_fichier):
                fichiers.append(chemin_fichier)
                labels.append(classe)

# 2. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    fichiers, labels, test_size=RATIO_TEST, stratify=labels, random_state=42
)

# 3. Copier les fichiers dans une nouvelle structure train/test
for split_name, X, y in [('train', X_train, y_train), ('test', X_test, y_test)]:
    for chemin_source, classe in zip(X, y):
        dossier_cible = os.path.join(DOSSIER_DEST, split_name, classe)
        os.makedirs(dossier_cible, exist_ok=True)
        shutil.copy(chemin_source, os.path.join(dossier_cible, os.path.basename(chemin_source)))

print("✅ Split terminé avec succès !")

✅ Split terminé avec succès !


In [3]:
### YOLO
!pip install ultralytics

  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)


Pour voir, utilisation du code fourni par ultralytics pour personnaliser l'entraînement, avec commentaire et adaptation de GPT pour m'aider à comprendre torch

In [ ]:
import torch
import torchvision.transforms as T

from ultralytics import YOLO
from ultralytics.data.dataset import ClassificationDataset
from ultralytics.models.yolo.classify import ClassificationTrainer
from ultralytics.models.yolo.classify import ClassificationValidator  # N'oublie pas d'importer cela aussi

# -----------------------------
# 💾 Définition du Dataset personnalisé
# -----------------------------
class CustomizedDataset(ClassificationDataset):
    """
    Dataset personnalisé pour la classification avec des transformations spécifiques à l'entraînement et à la validation.
    Hérite de ClassificationDataset d'Ultralytics.
    """

    def __init__(self, root: str, args, augment: bool = False, prefix: str = ""):
        # Initialise le dataset parent (chargement des fichiers, structure de dossier, etc.)
        super().__init__(root, args, augment, prefix)

        # 🧪 Transformations utilisées pendant l'entraînement
        train_transforms = T.Compose([
            T.Resize((args.imgsz, args.imgsz)),  # Redimensionnement uniforme
            T.RandomHorizontalFlip(p=args.fliplr),  # Flip horizontal
            T.RandomVerticalFlip(p=args.flipud),    # Flip vertical
            T.RandAugment(interpolation=T.InterpolationMode.BILINEAR),  # Augmentation aléatoire
            T.ColorJitter(  # Changements aléatoires de couleur
                brightness=args.hsv_v,
                contrast=args.hsv_v,
                saturation=args.hsv_s,
                hue=args.hsv_h
            ),
            T.ToTensor(),  # Conversion en tenseur PyTorch
            T.Normalize(mean=[0.5], std=[0.5]),  # Normalisation (adaptée si images en niveaux de gris ou RGB)
            T.RandomErasing(p=args.erasing, inplace=True),  # Supprime des parties aléatoires de l’image
        ])

        # 🧪 Transformations pour la validation (plus simples, pas d'augmentation)
        val_transforms = T.Compose([
            T.Resize((args.imgsz, args.imgsz)),
            T.ToTensor(),
            T.Normalize(mean=[0.5], std=[0.5]),
        ])

        # On affecte la bonne transformation selon le mode (train/val)
        self.torch_transforms = train_transforms if augment else val_transforms


# -----------------------------
# 🚂 Entraîneur personnalisé
# -----------------------------
class CustomizedTrainer(ClassificationTrainer):
    """
    Entraîneur personnalisé qui utilise notre dataset avec les transformations définies ci-dessus.
    """

    def build_dataset(self, img_path: str, mode: str = "train", batch=None):
        return CustomizedDataset(
            root=img_path,
            args=self.args,
            augment=(mode == "train"),  # Appliquer l'augmentation uniquement à l'entraînement
            prefix=mode
        )


# -----------------------------
# 📊 Validateur personnalisé
# -----------------------------
class CustomizedValidator(ClassificationValidator):
    """
    Validateur personnalisé utilisant les mêmes transformations que celles prévues pour la validation.
    """

    def build_dataset(self, img_path: str, mode: str = "train"):
        return CustomizedDataset(
            root=img_path,
            args=self.args,
            augment=(mode == "train"),
            prefix=self.args.split
        )


# -----------------------------
# 📦 Chargement du modèle et entraînement
# -----------------------------
model = YOLO("yolo11s-cls.pt")  # ← Choisis un modèle de classification YOLO pré-entraîné

# Démarre l'entraînement avec le CustomizedTrainer
model.train(
    data="/kaggle/working/split",  # Dossier contenant les sous-dossiers par classe
    trainer=CustomizedTrainer,
    epochs=100,
    imgsz=224,
    batch=64,
    device='0',
    amp = True,
    workers = 8
    
)

# Évaluation avec le validateur personnalisé
model.val(
    data="/kaggle/working/split",
    validator=CustomizedValidator,
    imgsz=224,
    batch=64,
     device='0',
    amp = True,
    workers = 8
    
)

Ultralytics 8.3.161 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/split, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

train: Scanning /kaggle/working/split/train... 876 images, 0 corrupt: 100%|██████████| 876/876 [00:00<00:00, 2799.81it/s]

train: New cache created: /kaggle/working/split/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1156.0±1148.7 MB/s, size: 548.0 KB)


val: Scanning /kaggle/working/split/test... 220 images, 0 corrupt: 100%|██████████| 220/220 [00:00<00:00, 1877.99it/s]

...

               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00, 14.52it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all      0.391      0.845
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
Ultralytics 8.3.161 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11s-cls summary (fused): 47 layers, 5,457,186 parameters, 0 gradients, 12.0 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /kaggle/working/split/train... found 876 images in 18 classes ✅ 
val: /kaggle/working/split/test... found 220 images in 18 classes ✅ 
test: /kaggle/working/split/test... found 220 images in 18 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3472.4±1606.5 MB/s, size: 548.0 KB)


val: Scanning /kaggle/working/split/test... 220 images, 0 corrupt: 100%|██████████| 220/220 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all      0.355      0.836
Speed: 1.0ms preprocess, 2.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train22


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b0699512ed0>
curves: []
curves_results: []
fitness: 0.595454528927803
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.3545454442501068, 'metrics/accuracy_top5': 0.8363636136054993, 'fitness': 0.595454528927803}
save_dir: PosixPath('runs/classify/train22')
speed: {'preprocess': 0.9991254636350856, 'inference': 2.2191947863647825, 'loss': 0.00034161818120992393, 'postprocess': 0.0005397863643255814}
task: 'classify'
top1: 0.3545454442501068
top5: 0.8363636136054993

In [ ]:
# Évaluation avec le validateur personnalisé

model.val(
    data="/kaggle/working/split",
    validator=CustomizedValidator,
    imgsz=224,
    batch=64,
     device='0',
    amp = True,
    workers = 8
)
# Affichage des résultats
print(model.results_dict())

NameError: name 'model' is not defined

In [ ]:
## regression multimodale avec embeddings d'image
